# Error Per Layered Gate (EPLG)

In [1]:
%load_ext autoreload
%autoreload 2

## Choose (or define) a backend

In [2]:
# from iqm.qiskit_iqm.iqm_provider import IQMProvider
#
# resonance_station="xxxxxxxxxxxxxxxxxxxx"
# iqm_url=f"https://cocos.resonance.meetiqm.com/{resonance_station}"
# cal_url=f"https://api.resonance.meetiqm.com/quantum-computers/v1/{resonance_station}/calibrations"
# provider = IQMProvider(iqm_url)
# backend = provider.get_backend()

backend="iqmfakeapollo"

## Set IQM Token if using Resonance.

In [3]:
# import os
# os.environ["IQM_TOKEN"] = ""

## Randomized Benchmarking Configuration

NB: Clifford RB is executed by default with Interleaved RB !

In [ ]:
from iqm.benchmarks.randomized_benchmarking.eplg.eplg import *

In [5]:
EXAMPLE_EPLG = EPLGConfiguration(
    custom_qubits_array=[[0,1],[1,4],[4,5],[5,6],[6,11],[11,16]],
    drb_depths=sorted(list(set(np.geomspace(1, 200, num=10, endpoint=True, dtype=int).tolist())), reverse=True),
    drb_circuit_samples=25,
    shots=2**8,
    chain_length=8,
    chain_path_samples=100,
    num_disjoint_layers=2,
)

## Run the experiment

In [6]:
benchmark_eplg = EPLGBenchmark(backend, EXAMPLE_EPLG)
run_eplg = benchmark_eplg.run()

2025-03-25 09:45:40,914 - iqm.benchmarks.logging_config - INFO - WARNING: custom_qubits_array validation not yet implemented! Make sure it forms a linear chain.
2025-03-25 09:45:48,470 - iqm.benchmarks.logging_config - INFO - Clifford dictionaries imported successfully !
2025-03-25 09:45:48,471 - iqm.benchmarks.logging_config - INFO - Executing parallel Direct RB on qubits [[0, 1], [4, 5], [6, 11]] (group 1/2). Will generate and submit all 25 DRB circuits for each depth [200, 111, 61, 34, 18, 10, 5, 3, 1]
2025-03-25 09:45:48,472 - iqm.benchmarks.logging_config - INFO - Depth 200
2025-03-25 09:46:42,323 - iqm.benchmarks.logging_config - INFO - Submitting batch with 25 circuits corresponding to qubits [0, 1, 4, 5, 6, 11]
2025-03-25 09:46:42,330 - iqm.benchmarks.logging_config - INFO - Job for depth 200 submitted successfully!
2025-03-25 09:46:42,456 - iqm.benchmarks.logging_config - INFO - Depth 111
2025-03-25 09:47:20,000 - iqm.benchmarks.logging_config - INFO - Submitting batch with 25

## Perform the analysis

In [7]:
result_eplg = benchmark_eplg.analyze()

2025-03-25 09:50:57,022 - iqm.benchmarks.logging_config - INFO - Post-processing parallel Direct RB on qubits [[0, 1], [4, 5], [6, 11]].
2025-03-25 09:50:57,039 - iqm.benchmarks.logging_config - INFO - Depth 200
2025-03-25 09:50:57,061 - iqm.benchmarks.logging_config - INFO - Depth 111
2025-03-25 09:50:57,083 - iqm.benchmarks.logging_config - INFO - Depth 61
2025-03-25 09:50:57,106 - iqm.benchmarks.logging_config - INFO - Depth 34
2025-03-25 09:50:57,127 - iqm.benchmarks.logging_config - INFO - Depth 18
2025-03-25 09:50:57,147 - iqm.benchmarks.logging_config - INFO - Depth 10
2025-03-25 09:50:57,167 - iqm.benchmarks.logging_config - INFO - Depth 5
2025-03-25 09:50:57,187 - iqm.benchmarks.logging_config - INFO - Depth 3
2025-03-25 09:50:57,205 - iqm.benchmarks.logging_config - INFO - Depth 1
2025-03-25 09:50:57,320 - iqm.benchmarks.logging_config - INFO - Post-processing parallel Direct RB on qubits [[1, 4], [5, 6], [11, 16]].
2025-03-25 09:50:57,338 - iqm.benchmarks.logging_config - IN

In [8]:
run_eplg.dataset.attrs.keys()

dict_keys(['session_timestamp', 'execution_timestamp', 'backend_configuration_name', 'backend_name', 'benchmark', 'shots', 'max_gates_per_batch', 'max_circuits_per_batch', 'calset_id', 'routing_method', 'physical_layout', 'use_dd', 'dd_strategy', 'qubits_array', 'is_eplg', 'parallel_execution', 'depths', 'num_circuit_samples', 'qiskit_optim_level', 'two_qubit_gate_ensembles', 'densities_2q_gates', 'clifford_sqg_probabilities', 'sqg_gate_ensembles', 'parallel_all_0', 0, 'parallel_all_1', 1, 'backend', 'drb_depths', 'drb_circuit_samples', 'custom_qubits_array', 'chain_length', 'chain_path_samples', 'num_disjoint_layers', 'calibration_url'])

In [9]:
for o in result_eplg.observations:
    if o.name != "EPLG":
        print(f"{o.name} {o.identifier.qubit_indices}: {100*o.value:.2f} +/- {100*o.uncertainty:.2f} %")
    else:
        print(f"\n{o.name} {o.identifier.qubit_indices}: {o.value:.2e} +/- {o.uncertainty:.2e}")

average_gate_fidelity [0, 1]: 96.25 +/- 0.11 %
average_gate_fidelity [4, 5]: 96.36 +/- 0.08 %
average_gate_fidelity [6, 11]: 95.92 +/- 0.09 %
average_gate_fidelity [1, 4]: 96.42 +/- 0.11 %
average_gate_fidelity [5, 6]: 95.66 +/- 0.15 %
average_gate_fidelity [11, 16]: 95.74 +/- 0.12 %
Layer Fidelity (n_qubits=8): 78.55 +/- 0.22 %

EPLG (n_qubits=8): 3.94e-02 +/- 4.55e-04
